In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Trainer (has labels)
# trainer_df = pd.read_csv('datasets/cleaned_SOLIDtest6K_trainer.tsv', sep="\t")
# learner_tweets_df = pd.read_csv('datasets/cleaned_SOLID9M_learner.tsv', sep="\t")

tr_tweets_df = pd.read_csv('datasets/cleaned_tr_offenseval.tsv', sep="\t")


In [2]:
import numpy as np

tr_tweets = tr_tweets_df['tweet'].values
tr_labels = tr_tweets_df['label'].values


In [3]:
import torch
from torch.utils.data import DataLoader, Dataset


class TweetDataset(Dataset):
    def __init__(self, texts, labels=None):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        if self.labels is not None:
            label = self.labels[idx]
            return {
                'text': text,
                'label': label
            }
        else:
            return {
                'text': text
            }
        
     
# trainer_dataset = TweetDataset(trainer_tweets, trainer_labels)
train_dataset = TweetDataset(tr_tweets, tr_labels)

# trainer_loader = DataLoader(trainer_dataset, batch_size=12, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=6, shuffle=False)


In [4]:
import torch
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
from torch import optim
from tqdm import tqdm

num_epochs = 1

tokenizer = DistilBertTokenizer.from_pretrained('distilbert/distilbert-base-uncased')


model = DistilBertForSequenceClassification.from_pretrained("models/SOLIDOLID_80k_DISTILBERT_1", num_labels=2)


optimizer = optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)


/home/cymn/miniconda3/envs/nvidia-newest/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [5]:
model.train()
for epoch in range(num_epochs):
    # trainer_dataloader_iterator = iter(trainer_loader)
    semi_labeled_dataloader_iterator = iter(train_loader)
    num_batches = len(train_loader)
    # num_batches = 5000
    for _ in tqdm(range(num_batches)):
        # Train on labeled data
        batch = next(semi_labeled_dataloader_iterator, None)
        if batch is not None:
            inputs = tokenizer(batch['text'], padding=True, truncation=True, max_length=512,return_tensors="pt").to(device)
            labels = torch.tensor(batch['label']).clone().detach().to(device)
            outputs = model(**inputs)
            logits = outputs.logits
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

  0%|          | 0/5213 [00:00<?, ?it/s]/tmp/ipykernel_50006/1278718171.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['label']).clone().detach().to(device)
100%|██████████| 5213/5213 [03:48<00:00, 22.79it/s]


In [6]:
model.save_pretrained('models/troffenseval_SOLIDdistilBERT')